## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

Aug. 4 Aug. 10


In [17]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [18]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [19]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [20]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [21]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,August-5-2025,Condo/Co-op,318 Flamingo Ln,Delray Beach,FL,33445.0,265000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/318-Fla...,Beaches MLS,RX-10990471,N,Y,26.474002,-80.117183


In [8]:
print('Input county name:')
county = input()

Input county name:


 Palm Beach County


In [22]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [23]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [24]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [25]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-08-08    26
2025-08-05    20
2025-08-06    18
2025-08-07    12
Name: count, dtype: int64

In [26]:
df2 = df2.sort_values(by='PRICE',ascending=False)

## Collect Agent Information

In [27]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }
        response = requests.get(base_url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()

        # Safely drill down, falling back to {} if any level is None or missing
        info = data.get('data') or {}
        panel = info.get('mainHouseInfoPanelInfo') or {}
        house = panel.get('mainHouseInfo') or {}

        # Extract listing agents
        listing_agents = house.get('listingAgents') or []
        list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName') if len(listing_agents) > 0 else None
        list_broker_name_1 = listing_agents[0].get('brokerName')           if len(listing_agents) > 0 else None
        list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName') if len(listing_agents) > 1 else None
        list_broker_name_2 = listing_agents[1].get('brokerName')           if len(listing_agents) > 1 else None

        # Extract buying agents
        buying_agents = house.get('buyingAgents') or []
        buy_agent_name_1  = buying_agents[0].get('agentInfo', {}).get('agentName') if len(buying_agents) > 0 else None
        buy_broker_name_1 = buying_agents[0].get('brokerName')             if len(buying_agents) > 0 else None
        buy_agent_name_2  = buying_agents[1].get('agentInfo', {}).get('agentName') if len(buying_agents) > 1 else None
        buy_broker_name_2 = buying_agents[1].get('brokerName')             if len(buying_agents) > 1 else None

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return { key: None for key in [
            'url', 'list_agent_name_1','list_broker_name_1',
            'list_agent_name_2','list_broker_name_2',
            'buy_agent_name_1','buy_broker_name_1',
            'buy_agent_name_2','buy_broker_name_2'
        ] }

In [28]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [29]:
response_df = pd.DataFrame(response_list)

In [30]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [31]:
df_top_ten = merged_df

In [32]:
df_top_ten

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,7,PAST SALE,2025-08-06,Condo/Co-op,2000 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,4300000.0,3.0,...,-80.037955,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,John Cregan,"Sotheby's Intl. Realty, Inc.",Lisa Cregan,"Sotheby's Intl. Realty, Inc.",Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None
1,4,PAST SALE,2025-08-06,Condo/Co-op,700 Ocean Royale 1201 Way #1201,Juno Beach,FL,33408.0,2750000.0,3.0,...,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Rose Vasha,Illustrated Properties LLC (Co,None,None,Rose Vasha,Illustrated Properties LLC (Co,None,None
2,50,PAST SALE,2025-08-07,Condo/Co-op,227 Brazilian Ave Ph D,Palm Beach,FL,33480.0,2350000.0,3.0,...,-80.037508,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Alice Hodach,Keller Williams Palm Beach Isl,None,None,Alice Hodach,Keller Williams Palm Beach Isl,None,None
3,2,PAST SALE,2025-08-08,Condo/Co-op,701 S Olive Ave #1803,West Palm Beach,FL,33401.0,2150000.0,2.0,...,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Christopher Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Christopher Cronican,Douglas Elliman
4,40,PAST SALE,2025-08-08,Condo/Co-op,790 Andrews Ave Unit C204,Delray Beach,FL,33483.0,1310000.0,2.0,...,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Charles Luciano,Compass Florida LLC,None,None,CLAUDIA FISHER CLAUDIA FISHER TEAM,Douglas Elliman,None,None
5,45,PAST SALE,2025-08-05,Condo/Co-op,550 S Ocean Blvd #906,Boca Raton,FL,33432.0,1250000.0,2.0,...,-80.070723,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Lisa Galante,Lang Realty/Delray Beach,None,None,Ana Londono,Lang Realty/BR,None,None
6,17,PAST SALE,2025-08-08,Condo/Co-op,17597 Ashbourne Way Unit A,Boca Raton,FL,33496.0,1025000.0,3.0,...,-80.128578,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Michael Ledwitz,Engel & Volkers Boca Raton,Wendy Ledwitz,Engel & Volkers Boca Raton,Michael Ledwitz,Engel & Volkers Boca Raton,None,None
7,43,PAST SALE,2025-08-08,Condo/Co-op,4511 S Ocean Blvd #804,Highland Beach,FL,33487.0,755000.0,2.0,...,-80.065805,https://www.redfin.com/FL/Highland-Beach/4511-...,Mary Sacks,Champagne & Parisi Real Estate,None,None,Adrienne Marks,Compass Florida LLC,None,None
8,11,PAST SALE,2025-08-07,Condo/Co-op,400 Ocean Trail Way #707,Jupiter,FL,33477.0,650000.0,2.0,...,-80.071033,https://www.redfin.com/FL/Jupiter/400-Ocean-Tr...,Maria Rosado,"Crane Reed Properties, LLC",Courtney Smitheman,"Crane Reed Properties, LLC",Sean Underwood,Jupiter By the Sea Realty Inc,Michel Underwood,Jupiter By the Sea Realty Inc
9,61,PAST SALE,2025-08-06,Condo/Co-op,3400 S Ocean Blvd Unit PHL,Highland Beach,FL,33487.0,630000.0,2.0,...,-80.065371,https://www.redfin.com/FL/Highland-Beach/3400-...,Christine Balistreri Vanbuskirk,Balistreri Real Estate Inc,Laura Balistreri,Balistreri Real Estate Inc,Christine Balistreri Vanbuskirk,Balistreri Real Estate Inc,None,None


## Current Week's Values

In [33]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

76


In [34]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$413,454


In [35]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$278


In [36]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$31,422,500


In [37]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           76              413454          278                     31422500.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [38]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
94
-----------
Input Previous Week Condo Average Sales Price:
395110
-----------
Input Previous Week Condo Average PSF:
267
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
37140300.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [39]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [40]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [41]:
df_top_ten.columns

Index(['index', 'SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [42]:
pd.set_option('display.max_columns',None)

In [43]:
df_top_ten.head(1)

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,7,PAST SALE,2025-08-06,Condo/Co-op,2000 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,4300000.0,3.0,3.5,Two Thousand Condo,3262.0,NaN,1981.0,NaN,1318.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,Beaches MLS,RX-11091385,N,Y,26.644359,-80.037955,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,John Cregan,"Sotheby's Intl. Realty, Inc.",Lisa Cregan,"Sotheby's Intl. Realty, Inc.",Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange


In [44]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [45]:
df_top_ten

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,7,PAST SALE,2025-08-06,Condo/Co-op,2000 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,4300000.0,3.0,3.5,Two Thousand Condo,3262.0,NaN,1981.0,NaN,1318.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,Beaches MLS,RX-11091385,N,Y,26.644359,-80.037955,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,John Cregan,"Sotheby's Intl. Realty, Inc.",Lisa Cregan,"Sotheby's Intl. Realty, Inc.",Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange
1,4,PAST SALE,2025-08-06,Condo/Co-op,700 Ocean Royale 1201 Way #1201,Juno Beach,FL,33408.0,2750000.0,3.0,3.5,Ocean Royale Condo,2936.0,NaN,2000.0,NaN,937.0,3133.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Beaches MLS,RX-11050728,N,Y,26.891690,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Rose Vasha,Illustrated Properties LLC (Co,None,None,Rose Vasha,Illustrated Properties LLC (Co,None,None,2,blue
2,50,PAST SALE,2025-08-07,Condo/Co-op,227 Brazilian Ave Ph D,Palm Beach,FL,33480.0,2350000.0,3.0,2.0,Brazilian OF Palm Beach Condo LT 4 Blk E Royal...,1854.0,NaN,1970.0,NaN,1268.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Beaches MLS,RX-11080163,N,Y,26.704731,-80.037508,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Alice Hodach,Keller Williams Palm Beach Isl,None,None,Alice Hodach,Keller Williams Palm Beach Isl,None,None,3,blue
3,2,PAST SALE,2025-08-08,Condo/Co-op,701 S Olive Ave #1803,West Palm Beach,FL,33401.0,2150000.0,2.0,2.5,Two City Plaza Condo,2352.0,NaN,2008.0,NaN,914.0,2350.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11112095,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Christopher Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Christopher Cronican,Douglas Elliman,4,blue
4,40,PAST SALE,2025-08-08,Condo/Co-op,790 Andrews Ave Unit C204,Delray Beach,FL,33483.0,1310000.0,2.0,2.0,Landings OF Delray Beach Condo,1848.0,NaN,1990.0,NaN,709.0,1433.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11075041,N,Y,26.473356,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Charles Luciano,Compass Florida LLC,None,None,CLAUDIA FISHER CLAUDIA FISHER TEAM,Douglas Elliman,None,None,5,blue
5,45,PAST SALE,2025-08-05,Condo/Co-op,550 S Ocean Blvd #906,Boca Raton,FL,33432.0,1250000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,863.0,1524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11086188,N,Y,26.344456,-80.070723,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Lisa Galante,Lang Realty/Delray Beach,None,None,Ana Londono,Lang Realty/BR,None,None,6,blue
6,17,PAST SALE,2025-08-08,Condo/Co-op,17597 Ashbourne Way Unit A,Boca Raton,FL,33496.0,1025000.0,3.0,2.5,Ashbourne / Polo Club,2232.0,NaN,1987.0,NaN,459.0,965.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Beaches MLS,RX-11082164,N,Y,26.415867,-80.128578,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Michael Ledwitz,Engel & Volkers Boca Raton,Wendy Ledwitz,Engel & Volkers Boca Raton,Michael Ledwitz,Engel & Volkers Boca Raton,None,None,7,blue
7,43,PAST SALE,2025-08-08,Condo/Co-op,4511 S Ocean Blvd #804,Highland Beach,FL,33487.0,755000.0,2.0,2.0,45 Ocean Condo,1100.0,NaN,1974.0,NaN,686.0,1425.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4511-...,Beaches MLS,RX-11088587,N,Y,26.395664,-80.065805,https://www.redfin.com/FL/Highland-Beach/4511-...,Mary Sacks,Champagne & Parisi Real Estate,None,None,Adrienne Marks,Co

In [46]:
%store -r map_box_api_key

In [47]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [48]:
m.save('index.html')

## Data snagger

In [49]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [50]:
df_top_ten.head(1)

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,7,PAST SALE,2025-08-06,Condo/Co-op,2000 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,4300000.0,3.0,3.5,Two Thousand Condo,3262.0,NaN,1981.0,NaN,1318.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,Beaches MLS,RX-11091385,N,Y,26.644359,-80.037955,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,John Cregan,"Sotheby's Intl. Realty, Inc.",Lisa Cregan,"Sotheby's Intl. Realty, Inc.",Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [52]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_08112025


# CREATE TEMPLATE 

In [53]:
muni_set = set(df_top_ten['CITY'])

In [54]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [55]:
df_top_ten.reset_index(inplace=True,drop=True)

In [56]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [57]:
top_sale

'Two Thousand Condo at 2000 S Ocean Blvd Unit 501n in Palm Beach'

In [58]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [59]:
df_top_ten.columns

Index(['index', 'SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [60]:
df_top_ten

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,7,PAST SALE,2025-08-06,Condo/Co-op,2000 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,4300000.0,3.0,3.5,Two Thousand Condo,3262.0,NaN,1981.0,NaN,1318.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,Beaches MLS,RX-11091385,N,Y,26.644359,-80.037955,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,John Cregan,"Sotheby's Intl. Realty, Inc.",Lisa Cregan,"Sotheby's Intl. Realty, Inc.",Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange,Two Thousand Condo at 2000 S Ocean Blvd Unit 5...
1,4,PAST SALE,2025-08-06,Condo/Co-op,700 Ocean Royale 1201 Way #1201,Juno Beach,FL,33408.0,2750000.0,3.0,3.5,Ocean Royale Condo,2936.0,NaN,2000.0,NaN,937.0,3133.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Beaches MLS,RX-11050728,N,Y,26.891690,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Rose Vasha,Illustrated Properties LLC (Co,None,None,Rose Vasha,Illustrated Properties LLC (Co,None,None,2,blue,Ocean Royale Condo at 700 Ocean Royale 1201 Wa...
2,50,PAST SALE,2025-08-07,Condo/Co-op,227 Brazilian Ave Ph D,Palm Beach,FL,33480.0,2350000.0,3.0,2.0,Brazilian OF Palm Beach Condo LT 4 Blk E Royal...,1854.0,NaN,1970.0,NaN,1268.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Beaches MLS,RX-11080163,N,Y,26.704731,-80.037508,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Alice Hodach,Keller Williams Palm Beach Isl,None,None,Alice Hodach,Keller Williams Palm Beach Isl,None,None,3,blue,Brazilian OF Palm Beach Condo LT 4 Blk E Royal...
3,2,PAST SALE,2025-08-08,Condo/Co-op,701 S Olive Ave #1803,West Palm Beach,FL,33401.0,2150000.0,2.0,2.5,Two City Plaza Condo,2352.0,NaN,2008.0,NaN,914.0,2350.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11112095,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Christopher Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Christopher Cronican,Douglas Elliman,4,blue,Two City Plaza Condo at 701 S Olive Ave #1803 ...
4,40,PAST SALE,2025-08-08,Condo/Co-op,790 Andrews Ave Unit C204,Delray Beach,FL,33483.0,1310000.0,2.0,2.0,Landings OF Delray Beach Condo,1848.0,NaN,1990.0,NaN,709.0,1433.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11075041,N,Y,26.473356,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Charles Luciano,Compass Florida LLC,None,None,CLAUDIA FISHER CLAUDIA FISHER TEAM,Douglas Elliman,None,None,5,blue,Landings OF Delray Beach Condo at 790 Andrews ...
5,45,PAST SALE,2025-08-05,Condo/Co-op,550 S Ocean Blvd #906,Boca Raton,FL,33432.0,1250000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,863.0,1524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11086188,N,Y,26.344456,-80.070723,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Lisa Galante,Lang Realty/Delray Beach,None,None,Ana Londono,Lang Realty/BR,None,None,6,blue,Chalfonte Condo at 550 S Ocean Blvd #906 in Bo...
6,17,PAST SALE,2025-08-08,Condo/Co-op,17597 Ashbourne Way Unit A,Boca Raton,FL,33496.0,1025000.0,3.0,2.5,Ashbourne / Polo Club,2232.0,NaN,1987.0,NaN,459.0,965.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Beaches MLS,RX-11082164,N,Y,26.415867,-80.128578,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Michael Ledwitz,Engel & Volkers Boca Raton,Wendy Ledwitz,Engel & Volkers Boca Raton,Michael Ledwitz,Engel & Volkers Boca Raton,None,None,7,blue,Ashbourne / Polo Club at 17597 Ashbourne Way U...
7,43,PAST SALE,2025-08-08

In [61]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name.strip()}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Two Thousand Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $630,000 to $4,300,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Delray Beach, Juno Beach, Boca Raton, Jupiter, West Palm Beach, Highland Beach, Palm Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 76 condo sales totaling $31,422,500 million from Aug. 4 to Aug. 10. The previous week, brokers closed 94 condo sales totaling $37,140,300.

Last week’s units sold for 

In [62]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [63]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [64]:
df_top_ten.columns

Index(['index', 'SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [65]:
df_top_ten

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,7,PAST SALE,2025-08-06,Condo/Co-op,2000 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,4300000.0,3.0,3.5,Two Thousand Condo,3262.0,NaN,1981.0,NaN,1318.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,Beaches MLS,RX-11091385,N,Y,26.644359,-80.037955,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,John Cregan,"Sotheby's Intl. Realty, Inc.",Lisa Cregan,"Sotheby's Intl. Realty, Inc.",Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange,Two Thousand Condo at 2000 S Ocean Blvd Unit 5...
1,4,PAST SALE,2025-08-06,Condo/Co-op,700 Ocean Royale 1201 Way #1201,Juno Beach,FL,33408.0,2750000.0,3.0,3.5,Ocean Royale Condo,2936.0,NaN,2000.0,NaN,937.0,3133.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Beaches MLS,RX-11050728,N,Y,26.891690,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Rose Vasha,Illustrated Properties LLC (Co,None,None,Rose Vasha,Illustrated Properties LLC (Co,None,None,2,blue,Ocean Royale Condo at 700 Ocean Royale 1201 Wa...
2,50,PAST SALE,2025-08-07,Condo/Co-op,227 Brazilian Ave Ph D,Palm Beach,FL,33480.0,2350000.0,3.0,2.0,Brazilian OF Palm Beach Condo LT 4 Blk E Royal...,1854.0,NaN,1970.0,NaN,1268.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Beaches MLS,RX-11080163,N,Y,26.704731,-80.037508,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Alice Hodach,Keller Williams Palm Beach Isl,None,None,Alice Hodach,Keller Williams Palm Beach Isl,None,None,3,blue,Brazilian OF Palm Beach Condo LT 4 Blk E Royal...
3,2,PAST SALE,2025-08-08,Condo/Co-op,701 S Olive Ave #1803,West Palm Beach,FL,33401.0,2150000.0,2.0,2.5,Two City Plaza Condo,2352.0,NaN,2008.0,NaN,914.0,2350.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11112095,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Christopher Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Christopher Cronican,Douglas Elliman,4,blue,Two City Plaza Condo at 701 S Olive Ave #1803 ...
4,40,PAST SALE,2025-08-08,Condo/Co-op,790 Andrews Ave Unit C204,Delray Beach,FL,33483.0,1310000.0,2.0,2.0,Landings OF Delray Beach Condo,1848.0,NaN,1990.0,NaN,709.0,1433.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11075041,N,Y,26.473356,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Charles Luciano,Compass Florida LLC,None,None,CLAUDIA FISHER CLAUDIA FISHER TEAM,Douglas Elliman,None,None,5,blue,Landings OF Delray Beach Condo at 790 Andrews ...
5,45,PAST SALE,2025-08-05,Condo/Co-op,550 S Ocean Blvd #906,Boca Raton,FL,33432.0,1250000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,863.0,1524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11086188,N,Y,26.344456,-80.070723,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Lisa Galante,Lang Realty/Delray Beach,None,None,Ana Londono,Lang Realty/BR,None,None,6,blue,Chalfonte Condo at 550 S Ocean Blvd #906 in Bo...
6,17,PAST SALE,2025-08-08,Condo/Co-op,17597 Ashbourne Way Unit A,Boca Raton,FL,33496.0,1025000.0,3.0,2.5,Ashbourne / Polo Club,2232.0,NaN,1987.0,NaN,459.0,965.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Beaches MLS,RX-11082164,N,Y,26.415867,-80.128578,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Michael Ledwitz,Engel & Volkers Boca Raton,Wendy Ledwitz,Engel & Volkers Boca Raton,Michael Ledwitz,Engel & Volkers Boca Raton,None,None,7,blue,Ashbourne / Polo Club at 17597 Ashbourne Way U...
7,43,PAST SALE,2025-08-08

In [66]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Two Thousand Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $630,000 to $4,300,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Delray Beach, Juno Beach, Boca Raton, Jupiter, West Palm Beach, Highland Beach, Palm Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 76 condo sales totaling $31,422,500 million from Aug. 4 to Aug. 10. The previous week, brokers closed 94 condo sales totaling $37,140,300.

Last week’s units sold for 

In [67]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Palm-Beach/2000-S-Ocean-Blvd-33480/unit-501N/home/42648936


In [68]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Juno-Beach/700-Ocean-Royale-Way-33408/unit-1201/home/42561683


In [69]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Highland-Beach/3400-S-Ocean-Blvd-33487/unit-PHL/home/42388632


In [70]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Jupiter/400-Ocean-Trail-Way-33477/unit-707/home/42409401


In [71]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/2000-S-Ocean-Blvd-33480/unit-501N/home/42648936


In [72]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Highland-Beach/3400-S-Ocean-Blvd-33487/unit-PHL/home/42388632


In [73]:
df_top_ten

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,7,PAST SALE,2025-08-06,Condo/Co-op,2000 S Ocean Blvd Unit 501n,Palm Beach,FL,33480.0,4300000.0,3.0,3.5,Two Thousand Condo,3262.0,NaN,1981.0,NaN,1318.0,4060.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,Beaches MLS,RX-11091385,N,Y,26.644359,-80.037955,https://www.redfin.com/FL/Palm-Beach/2000-S-Oc...,John Cregan,"Sotheby's Intl. Realty, Inc.",Lisa Cregan,"Sotheby's Intl. Realty, Inc.",Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange,Two Thousand Condo at 2000 S Ocean Blvd Unit 5...
1,4,PAST SALE,2025-08-06,Condo/Co-op,700 Ocean Royale 1201 Way #1201,Juno Beach,FL,33408.0,2750000.0,3.0,3.5,Ocean Royale Condo,2936.0,NaN,2000.0,NaN,937.0,3133.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Beaches MLS,RX-11050728,N,Y,26.891690,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Rose Vasha,Illustrated Properties LLC (Co,None,None,Rose Vasha,Illustrated Properties LLC (Co,None,None,2,blue,Ocean Royale Condo at 700 Ocean Royale 1201 Wa...
2,50,PAST SALE,2025-08-07,Condo/Co-op,227 Brazilian Ave Ph D,Palm Beach,FL,33480.0,2350000.0,3.0,2.0,Brazilian OF Palm Beach Condo LT 4 Blk E Royal...,1854.0,NaN,1970.0,NaN,1268.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Beaches MLS,RX-11080163,N,Y,26.704731,-80.037508,https://www.redfin.com/FL/Palm-Beach/227-Brazi...,Alice Hodach,Keller Williams Palm Beach Isl,None,None,Alice Hodach,Keller Williams Palm Beach Isl,None,None,3,blue,Brazilian OF Palm Beach Condo LT 4 Blk E Royal...
3,2,PAST SALE,2025-08-08,Condo/Co-op,701 S Olive Ave #1803,West Palm Beach,FL,33401.0,2150000.0,2.0,2.5,Two City Plaza Condo,2352.0,NaN,2008.0,NaN,914.0,2350.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11112095,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Christopher Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Lori Cronican,Douglas Elliman,Christopher Cronican,Douglas Elliman,4,blue,Two City Plaza Condo at 701 S Olive Ave #1803 ...
4,40,PAST SALE,2025-08-08,Condo/Co-op,790 Andrews Ave Unit C204,Delray Beach,FL,33483.0,1310000.0,2.0,2.0,Landings OF Delray Beach Condo,1848.0,NaN,1990.0,NaN,709.0,1433.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/790-And...,Beaches MLS,RX-11075041,N,Y,26.473356,-80.059371,https://www.redfin.com/FL/Delray-Beach/790-And...,Charles Luciano,Compass Florida LLC,None,None,CLAUDIA FISHER CLAUDIA FISHER TEAM,Douglas Elliman,None,None,5,blue,Landings OF Delray Beach Condo at 790 Andrews ...
5,45,PAST SALE,2025-08-05,Condo/Co-op,550 S Ocean Blvd #906,Boca Raton,FL,33432.0,1250000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,863.0,1524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11086188,N,Y,26.344456,-80.070723,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Lisa Galante,Lang Realty/Delray Beach,None,None,Ana Londono,Lang Realty/BR,None,None,6,blue,Chalfonte Condo at 550 S Ocean Blvd #906 in Bo...
6,17,PAST SALE,2025-08-08,Condo/Co-op,17597 Ashbourne Way Unit A,Boca Raton,FL,33496.0,1025000.0,3.0,2.5,Ashbourne / Polo Club,2232.0,NaN,1987.0,NaN,459.0,965.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Beaches MLS,RX-11082164,N,Y,26.415867,-80.128578,https://www.redfin.com/FL/Boca-Raton/17597-Ash...,Michael Ledwitz,Engel & Volkers Boca Raton,Wendy Ledwitz,Engel & Volkers Boca Raton,Michael Ledwitz,Engel & Volkers Boca Raton,None,None,7,blue,Ashbourne / Polo Club at 17597 Ashbourne Way U...
7,43,PAST SALE,2025-08-08

## Time on Market Calculator

In [75]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 1, 9) ## List (Earlier) date
date2 = datetime(2025, 8, 6) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

209


## Clean CSV for Datawrapper Chart

In [76]:
chart_df = df_top_ten

In [77]:
chart_df = chart_df.fillna(" ")

In [78]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [79]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [80]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [81]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [82]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [83]:
chart_df['PPSF'].astype(float)

0    1318.0
1     937.0
2    1268.0
3     914.0
4     709.0
5     863.0
6     459.0
7     686.0
8     556.0
9     434.0
Name: PPSF, dtype: float64

In [84]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [85]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [86]:
csv_date_string = today.strftime("%m_%d_%Y")

In [87]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [88]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-08-06,Two Thousand Condo,2000 S Ocean Blvd Unit 501n in Palm Beach,"$4,300,000","3,262","$1,318",1981,"John Cregan Sotheby's Intl. Realty, Inc. Lisa ...","Fern Fodiman Sotheby's Intl. Realty, Inc."
1,2025-08-06,Ocean Royale Condo,700 Ocean Royale 1201 Way #1201 in Juno Beach,"$2,750,000","2,936",$937,2000,Rose Vasha Illustrated Properties LLC (Co,Rose Vasha Illustrated Properties LLC (Co
2,2025-08-07,Brazilian OF Palm Beach Condo LT 4 Blk E Royal...,227 Brazilian Ave Ph D in Palm Beach,"$2,350,000","1,854","$1,268",1970,Alice Hodach Keller Williams Palm Beach Isl,Alice Hodach Keller Williams Palm Beach Isl
3,2025-08-08,Two City Plaza Condo,701 S Olive Ave #1803 in West Palm Beach,"$2,150,000","2,352",$914,2008,Christopher Cronican Douglas Elliman Lori Cron...,Lori Cronican Douglas Elliman Christopher Cron...
4,2025-08-08,Landings OF Delray Beach Condo,790 Andrews Ave Unit C204 in Delray Beach,"$1,310,000","1,848",$709,1990,Charles Luciano Compass Florida LLC,CLAUDIA FISHER CLAUDIA FISHER TEAM Douglas Ell...
5,2025-08-05,Chalfonte Condo,550 S Ocean Blvd #906 in Boca Raton,"$1,250,000","1,448",$863,1974,Lisa Galante Lang Realty/Delray Beach,Ana Londono Lang Realty/BR
6,2025-08-08,Ashbourne / Polo Club,17597 Ashbourne Way Unit A in Boca Raton,"$1,025,000","2,232",$459,1987,Michael Ledwitz Engel & Volkers Boca Raton Wen...,Michael Ledwitz Engel & Volkers Boca Raton
7,2025-08-08,45 Ocean Condo,4511 S Ocean Blvd #804 in Highland Beach,"$755,000","1,100",$686,1974,Mary Sacks Champagne & Parisi Real Estate,Adrienne Marks Compass Florida LLC
8,2025-08-07,Ocean Trail Condo IV,400 Ocean Trail Way #707 in Jupiter,"$650,000","1,170",$556,1983,"Maria Rosado Crane Reed Properties, LLC Courtn...",Sean Underwood Jupiter By the Sea Realty Inc M...
9,2025-08-06,Coronado At Highlandbeach,3400 S Ocean Blvd Unit PHL in Highland Beach,"$630,000","1,452",$434,1983,Christine Balistreri Vanbuskirk Balistreri Rea...,Christine Balistreri Vanbuskirk Balistreri Rea...
